In [4]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# 准备数据

In [5]:
(x,y),(x_test,y_test) = keras.datasets.mnist.load_data()

In [6]:
def proccess(x,y):
    x = tf.convert_to_tensor(tf.cast(x,dtype=tf.float32))/255.
    y = tf.convert_to_tensor(tf.cast(y,dtype=tf.int32))
    x = tf.reshape(x,[28*28])
    y = tf.one_hot(y,depth=10)
    return x,y

In [7]:
banchsize = 128

In [8]:
train_db = tf.data.Dataset.from_tensor_slices((x,y)).map(proccess).shuffle(10000).batch(banchsize)
test_db = tf.data.Dataset.from_tensor_slices((x,y)).map(proccess).batch(banchsize)

In [9]:
# 设置自定义层

In [12]:
class MyDense(keras.layers.Layer):
    def __init__(self,inp_dim,outp_dim):
        super(MyDense, self).__init__()
        
        self.kernel = self.add_weight('w', [inp_dim, outp_dim])
        self.bias = self.add_weight('b', [outp_dim])
        
    def call(self, inputs, training=None):   #重写call方法

        out = inputs @ self.kernel + self.bias

        return out 

In [13]:
class MyModel(keras.Model):  #继承keras.Model父类

    def __init__(self):    #继承父类属性
        super(MyModel, self).__init__()

        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)

    def call(self, inputs, training=None):   #重写call方法

        x = self.fc1(inputs)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x) 

        return x

In [14]:
network = MyModel()

In [17]:
network.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=tf.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

In [18]:
network.fit(train_db, epochs=10, validation_data=test_db,
              validation_freq=2)

Epoch 1/10
469/469 [==============================] - 3s 4ms/step - loss: 0.3096 - accuracy: 0.9063
Epoch 2/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1097 - accuracy: 0.9667 - val_loss: 0.0736 - val_accuracy: 0.9780
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0749 - accuracy: 0.9763
Epoch 4/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0522 - accuracy: 0.9834 - val_loss: 0.0446 - val_accuracy: 0.9859
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0413 - accuracy: 0.9875
Epoch 6/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0310 - accuracy: 0.9895 - val_loss: 0.0235 - val_accuracy: 0.9926
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0290 - accuracy: 0.9903
Epoch 8/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0240 - accuracy: 0.9919 - val_loss: 0.0162 - val_accuracy: 0.9948
Epoch 9/10
469/469 [============

In [19]:
network.evaluate(test_db)

469/469 [==============================] - 1s 2ms/step - loss: 0.0154 - accuracy: 0.9951


[0.015444026328623295, 0.995116651058197]

In [20]:
sample = next(iter(test_db))
x = sample[0]
y = sample[1] # one-hot
pred = network.predict(x) # [b, 10]
# convert back to number 
y = tf.argmax(y, axis=1)
pred = tf.argmax(pred, axis=1)

print(pred)
print(y)

4/4 [==============================] - 0s 2ms/step
tf.Tensor(
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 8 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9
 3 1 1 0 4 9 2 0 0 2 0 2 7 1 8 6 4], shape=(128,), dtype=int64)
tf.Tensor(
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9
 3 1 1 0 4 9 2 0 0 2 0 2 7 1 8 6 4], shape=(128,), dtype=int64)
